# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
# importing my libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor, plot_tree, plot_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import requests

In [ ]:
# import training data
df = pd.read_csv('/content/MyDrive/MyDrive/F1/f1_full_v3.csv')

# import testing data
test_data = pd.read_csv('/content/MyDrive/MyDrive/F1/predict.csv')

In [ ]:
# copying the data frame for the model
xgb2 = df.copy()

# Model

## Initialization

In [ ]:
# Columns to drop
cols_to_drop = ['driverid', 'driver', 'name', 'race_year', 'url']

# Drop the unwanted columns
xgb2.drop(columns=cols_to_drop, inplace = True)

# encode categorical columns
categorical_cols = ['circuit_type', 'direction', 'overtake_rating', 'track_type']
xgb2 = pd.get_dummies(xgb2, columns=categorical_cols, drop_first=True)

In [ ]:
# setup the testing and training split
# Define the target variable and the features
y = xgb2['position']
X = xgb2.drop(columns=['position'])

# Split the data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Optimization

### Hypertuning best params

In [ ]:
# Define an expanded parameter distribution for XGBoost
param_distributions = {
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_child_weight': [1, 2, 3, 4, 5],
    'gamma': [0, 0.01, 0.1, 0.2, 0.3],              # Minimum loss reduction required to make a split
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],    # Fraction of features used per tree
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],           # Fraction of samples used per tree
    'n_estimators': [100, 200, 300, 500],             # Number of trees
    'reg_alpha': [0, 0.01, 0.1, 1, 10],               # L1 regularization
    'reg_lambda': [1, 1.5, 2, 3, 5]                   # L2 regularization
}

# Initialize the XGBoost regressor with categorical support and the histogram tree method
xgb_model = XGBRegressor(enable_categorical=True, tree_method='hist')

# Setup RandomizedSearchCV:
# n_iter defines how many parameter settings are randomly sampled.
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=100,                 # Increase iterations to explore the expanded search space
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=2,
    random_state=42             # For reproducibility
)

# Assuming X_train and y_train are already defined from your train-test split:
random_search.fit(X_train, y_train)

# Retrieve the best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_
print("Best Parameters:", best_params)
print("Best CV Score (neg_mean_absolute_error):", best_score)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'subsample': 0.8, 'reg_lambda': 3, 'reg_alpha': 10, 'n_estimators': 500, 'min_child_weight': 4, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0.01, 'colsample_bytree': 0.7}
Best CV Score (neg_mean_absolute_error): -1.6968443632125854


### Training Model with best params

In [ ]:
# Define static parameters for XGBoost
params = {
  'subsample': 0.8,
  'reg_lambda': 3,
  'reg_alpha': 10,
  'n_estimators': 500,
  'min_child_weight': 4,
  'max_depth': 6,
  'learning_rate': 0.05,
  'gamma': 0.01,
  'colsample_bytree': 0.7}

# Initialize the XGBoost regressor with the static hyperparameters
xgb_model = XGBRegressor(
    enable_categorical=True,
    tree_method='hist',
    colsample_bytree=params['colsample_bytree'],
    learning_rate=params['learning_rate'],
    max_depth=params['max_depth'],
    min_child_weight=params['min_child_weight'],
    n_estimators=params['n_estimators'],
    subsample=params['subsample'])


# Train the model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared Score:", r2)

Mean Absolute Error (MAE): 1.7657370567321777
Mean Squared Error (MSE): 5.314754486083984
Root Mean Squared Error (RMSE): 2.305375129145794
R-squared Score: 0.8122528195381165


### Feature importance

In [ ]:
# Plot feature importance using XGBoost's built-in function
plt.figure(figsize=(10, 8))
plot_importance(xgb_model)
plt.title("Feature Importances")
plt.show()

importances = xgb_model.feature_importances_
features = X.columns

# Sort features by importance (highest first)
sorted_features = sorted(zip(features, importances), key=lambda x: x[1], reverse=True)

print("Sorted Feature Importances:")
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")

### Testing Model

In [ ]:
# Keep the driver information in a separate DataFrame or Series
drivers = test_data['driver']

# Assuming you previously dropped 'driver' before training the model
X_test = test_data.drop(columns=['driver'])

# Check if the 'finish' column is present and needs to be dropped for prediction
if 'finish' in X_test.columns:
    X_test = X_test.drop(columns='finish')

# Reset the column ordering
X_test = X_test[X.columns]

# Predict using the trained model
predictions = xgb_model.predict(X_test)

# Combine the driver information with the predictions
results = pd.DataFrame({
    'Driver': drivers,
    'Predicted_Finish': predictions
})

# Sort the DataFrame by the 'Predicted_Finish' column in ascending order
results_sorted = results.sort_values(by='Predicted_Finish', ascending=True)

# Print the sorted DataFrame
print(results_sorted)

 columns: ['grid', 'laps', 'length', 'turns', 'drs_zones', 'pit_stops', 'laps_led', 'is_verstappen', 'driver_rank', 'constructor_rank', 'average_speed', 'track_df', 'is_wet', 'traction', 'grip', 'abrasion', 'evolution', 'stress', 'braking', 'lateral', 'downforce', 'lap_down', 'dnf', 'circuit_type_1', 'circuit_type_2', 'circuit_type_3', 'circuit_type_4', 'direction_1', 'overtake_rating_2', 'overtake_rating_3', 'overtake_rating_4', 'track_type_Semi-Permanent', 'track_type_Street Circuit']
 columns: ['grid', 'laps', 'length', 'turns', 'drs_zones', 'pit_stops', 'laps_led', 'is_verstappen', 'driver_rank', 'constructor_rank', 'average_speed', 'track_df', 'is_wet', 'traction', 'grip', 'abrasion', 'evolution', 'stress', 'braking', 'lateral', 'downforce', 'lap_down', 'dnf', 'circuit_type_1', 'circuit_type_2', 'circuit_type_3', 'circuit_type_4', 'direction_1', 'overtake_rating_2', 'overtake_rating_3', 'overtake_rating_4', 'track_type_Semi-Permanent', 'track_type_Street Circuit']
                

# Storage

## Alternate Params

In [ ]:
'subsample': 0.6, 'reg_lambda': 3, 'reg_alpha': 0, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.03, 'gamma': 0.2, 'colsample_bytree': 0.8}